## 交通手段を用いた移動時間と徒歩の時間を考慮する

### 同じ路線のみ, 指定の時間以下で辿り着けるバス停を表示

In [19]:
from typing import Optional

from geopandas import GeoSeries
from shapely.lib import unary_union

from engine import Station, TransitType, get_route_yahoo_transit
from engine.bus import *
from engine.train import *

# データの読み込み
dataset: dict[TransitType, list[Station]] = {
    TransitType.BUS: load_stop_data("../dataset/busstops/kanagawa/P11-22_14.geojson"),
    TransitType.TRAIN: load_station_data("../dataset/stations/N02-20_Station.geojson"),
}


def _include(a: list[str], b: list[str]) -> bool:
    for a_content in a:
        if a_content in b:
            return True
    return False


def get_same_line_route_stations(station: Station) -> list[Station]:
    stations: list[Station] = []

    if station.transit_type == TransitType.BUS:
        for stop in dataset[TransitType.BUS]:
            if (
                    (stop.management_groups == station.management_groups)
                    and (_include(station.line_routes, stop.line_routes))
                    and (stop.name != station.name)
            ):
                stations.append(stop)
    elif station.transit_type == TransitType.TRAIN:
        for stat in dataset[TransitType.TRAIN]:
            if (stat.line_routes == station.line_routes) and (
                    stat.name != station.name
            ):
                stations.append(stat)

    return stations

def get_stations_with_time(from_: Station, limit_min: int) -> list[tuple[Station, int]]:
    stations: list[tuple[Station, int]] = []
    same_line_stations = get_same_line_route_stations(from_)
    transit_type = from_.transit_type
    
    for same_line_station in same_line_stations:
        will_add = False
        min_time_required = -1
        routes = get_route_yahoo_transit(transit_type, from_, same_line_station)
        # routeをすべてチェックする
        for route in routes:
            if (route["transfer"] == 0) and route["time_required"] <= limit_min:
                will_add = True
            # 最低所要時間を更新する
            if min_time_required == -1:
                min_time_required = route["time_required"]
            else:
                if route["time_required"] < min_time_required:
                    min_time_required = route["time_required"]
        if will_add:
            stations.append((same_line_station, min_time_required))
    
    return stations

# バスの本厚木駅を取り出す
honatugi_bus_stop: Optional[Station] = None
for stop in dataset[TransitType.BUS]:
    if stop.name == "本厚木駅":
        honatugi_bus_stop = stop
if honatugi_bus_stop is None:
    print("本厚木駅をみつけられませんでした")
    
res = get_stations_with_time(honatugi_bus_stop, 10)
for stat, time_req in res: # type: Station,int
    print(f"{time_req}分: {stat.name}")

10分: 松蓮寺
9分: 及川球技場入口
10分: 千頭
9分: 及川
9分: 妻田薬師
10分: そりだハイツ前
8分: 中村入口
8分: 穴口橋
6分: 妻田
6分: 妻田
7分: 林中学校入口
7分: 林
6分: 三家入口
6分: 戸室
10分: 緑ヶ丘小学校前
10分: 白山
6分: 吾妻団地
9分: 緑ヶ丘中央
5分: 木売場
9分: 小金
5分: 愛光病院前
5分: 戸室神社下
8分: 黄金原
4分: 市立病院前
3分: 松枝町一丁目
4分: 戸室住宅前
6分: アンリツ前
3分: 税務署入口
4分: 厚木警察署前
1分: 厚木東町
4分: 厚木高校前
1分: 小田急通り
1分: あつぎ大通り
2分: 厚木バスセンター
7分: 温水
3分: 栄町二丁目
8分: 温水児童館前
8分: 温水児童館前
4分: 恩名公民館前
8分: 高坪入口
7分: 恩名下
6分: 赤羽根中央
6分: 厚木市文化会館前
5分: 赤羽根入口


### 同じ路線のみ, 指定の時間以下で辿り着けるバス停, およびその周辺の徒歩エリア

In [20]:
from typing import Optional
from engine import Station, TransitType, get_route_yahoo_transit
from engine.bus import *
from engine.train import *

# データの読み込み
dataset: dict[TransitType, list[Station]] = {
    TransitType.BUS: load_stop_data("../dataset/busstops/kanagawa/P11-22_14.geojson"),
    TransitType.TRAIN: load_station_data("../dataset/stations/N02-20_Station.geojson"),
}


def _include(a: list[str], b: list[str]) -> bool:
    for a_content in a:
        if a_content in b:
            return True
    return False


def get_same_line_route_stations(station: Station) -> list[Station]:
    stations: list[Station] = []

    if station.transit_type == TransitType.BUS:
        for stop in dataset[TransitType.BUS]:
            if (
                    (stop.management_groups == station.management_groups)
                    and (_include(station.line_routes, stop.line_routes))
                    and (stop.name != station.name)
            ):
                stations.append(stop)
    elif station.transit_type == TransitType.TRAIN:
        for stat in dataset[TransitType.TRAIN]:
            if (stat.line_routes == station.line_routes) and (
                    stat.name != station.name
            ):
                stations.append(stat)

    return stations

def get_stations_with_time(from_: Station, limit_min: int) -> list[tuple[Station, int]]:
    stations: list[tuple[Station, int]] = []
    same_line_stations = get_same_line_route_stations(from_)
    transit_type = from_.transit_type

    for same_line_station in same_line_stations:
        will_add = False
        min_time_required = -1
        routes = get_route_yahoo_transit(transit_type, from_, same_line_station)
        # routeをすべてチェックする
        for route in routes:
            if (route["transfer"] == 0) and route["time_required"] <= limit_min:
                will_add = True
            # 最低所要時間を更新する
            if min_time_required == -1:
                min_time_required = route["time_required"]
            else:
                if route["time_required"] < min_time_required:
                    min_time_required = route["time_required"]
        if will_add:
            stations.append((same_line_station, min_time_required))

    return stations

# バスの本厚木駅を取り出す
honatugi_bus_stop: Optional[Station] = None
for stop in dataset[TransitType.BUS]:
    if stop.name == "本厚木駅":
        honatugi_bus_stop = stop
if honatugi_bus_stop is None:
    print("本厚木駅をみつけられませんでした")



In [63]:
from typing import Any
import geopandas


def concat_isochrones_per_contour(isochrones: list[dict[Any, Any]]) -> dict[int, dict]:
    result: dict[int, dict[Any, Any]] = {}  # int -> time_contour -> ["features"]["properties"]["contour"]
    for feature_collection in isochrones:
        gdf = geopandas.GeoDataFrame.from_features(feature_collection)
        for feature in gdf.iterfeatures():
            contour = feature["properties"]["contour"]
            # init
            if contour not in result:
                result[contour] = feature
            else:
                result[contour] = geopandas.GeoSeries(unary_union([result[contour], feature])).to_dict()
  
    return result

def concat_isochrones(isochrones: list[dict[Any, Any]]) -> str:
    gdf_geometries = []
    for feature_collection in isochrones:
        gdf = geopandas.GeoDataFrame.from_features(feature_collection)
        for gdf_geometry in gdf.geometry:
            gdf_geometries.append(gdf_geometry)
    return geopandas.GeoSeries(unary_union(gdf_geometries)).to_json()

In [64]:
import folium
import os
from dotenv import load_dotenv
from engine.mapbox import MapBoxApi, IsochroneProfile

load_dotenv()
api = MapBoxApi(os.getenv("MAPBOX_API_TOKEN"))

near_isochrones = []

max_min = 10
for near_station, time_req in res: # type: Station,int
    contour = max_min - time_req
    if contour < 1 or 60 < contour: # API側に1 <= contur <= 60の制限がある
        contour = 1
    isochrone_feature_collection = api.get_isochrone(
        prof=IsochroneProfile.Walking,
        coordinate=near_station.geometry.calc_mean(),
        contours_minutes=[contour]
    )
    near_isochrones.append(isochrone_feature_collection)

area_10mins_walking_and_bus_dict = concat_isochrones(near_isochrones)
print(area_10mins_walking_and_bus_dict)

m = folium.Map(
    location=[35.4861002, 139.3399782],
    zoom_start=14,
    tiles="https://cyberjapandata.gsi.go.jp/xyz/std/{z}/{x}/{y}.png",
    attr=f"出典: 国土地理院ウェブサイト・地理院タイル・標準地図 {'(C) MAPBOX'}",
)
folium.GeoJson(area_10mins_walking_and_bus_dict).add_to(m)

m



{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {}, "geometry": {"type": "Polygon", "coordinates": [[[139.350615, 35.469586], [139.349615, 35.470515], [139.348472, 35.469528], [139.348974, 35.468888], [139.350615, 35.468439], [139.350615, 35.469586]]]}, "bbox": [139.348472, 35.468439, 139.350615, 35.470515]}, {"id": "1", "type": "Feature", "properties": {}, "geometry": {"type": "Polygon", "coordinates": [[[139.344763, 35.467289], [139.343763, 35.468108], [139.342808, 35.468094], [139.342501, 35.467139], [139.343763, 35.466425], [139.344763, 35.466767], [139.344763, 35.467289]]]}, "bbox": [139.342501, 35.466425, 139.344763, 35.468108]}, {"id": "2", "type": "Feature", "properties": {}, "geometry": {"type": "Polygon", "coordinates": [[[139.337582, 35.467451], [139.336677, 35.466268], [139.336835, 35.465521], [139.337582, 35.46511], [139.338533, 35.465318], [139.338732, 35.466119], [139.338361, 35.467047], [139.337582, 35.467451]]]}, "bbox": [139.336